In [ ]:
from PIL import Image, ImageOps
from skimage import io, measure, metrics, color, transform
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import cv2
import numpy as np
import re
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# Split TIFF sequence into individual TIFF images
im = Image.open("path/to/file.tif")
for i in range(1, 28):
    try:
        im.seek(i)
        im.save(f"path/to/file{i}.tif")
    except EOFError:
        break

In [ ]:
# Rename files
all_masks = os.listdir("path/to/dir")
for i in all_masks:
    for p in range(5, 28):
        if i == "file04" + str(p) + "_mask_2.tif":
            os.rename("file04" + str(p) + "_mask_2.tif",
                      "file 04" + str(p) + "_mask_2.tif")
            
# Move files
for i in all_masks:
    for p in range(1, 28):
        if i == "file 04" + str(p) + "_mask_2.tif":
            os.rename("path/to/file 04" + str(p) + "_mask_2.tif",
                      "path2/to/file 04" + str(p) + "_mask_2.tif")

In [ ]:
# Merge two images
masks = os.listdir("path/to/dir")
masks2 = os.listdir("path/to/dir2")
masks.sort(key=lambda var: [int(x) if x.isdigit() else x
                            for x in re.findall(r"[^0-9]|[0-9]+", var)])
masks2.sort(key=lambda var: [int(x) if x.isdigit() else x
                             for x in re.findall(r"[^0-9]|[0-9]+", var)])

for i in range(len(masks)):
    first_image = cv2.imread("path/to/file/" + masks[i])
    second_image = cv2.imread("path/to/file/" + masks2[i])
    blended_img = cv2.add(first_image, second_image)
    cv2.imwrite("path/to/file/" + masks[i], blended_img)

In [ ]:
# Zero-pad images
val_masks = os.listdir("path/to/dir")
for i in val_masks:
    im = Image.open("path/to/file/" + i)
    old_size = im.size
    new_size = (1398, 1798)

    if old_size[0] % 2 == 0:
        add_left = add_right = (new_size[0] - old_size[0]) // 2
    else:
        add_left = (new_size[0] - old_size[0]) // 2
        add_right = (new_size[0] - old_size[0]) // 2 + 1
    
    if old_size[1] % 2 == 0:
        add_top = add_bottom = (new_size[1] - old_size[1]) // 2
    else:
        add_top = (new_size[1] - old_size[1]) // 2
        add_bottom = (new_size[1] - old_size[1]) // 2 + 1

    im_with_border = ImageOps.expand(im, border=(add_left, add_top, add_right, add_bottom))
    im_with_border.save("path/to/bordered_" + i)

In [ ]:
# Crop images
os.chdir("path/to/dir")
for i in os.listdir():
    if re.search(r"whirls 13", i) != None:
        im = cv2.imread(i, -1)
        cropped_im = im[:, 18:]
        cv2.imwrite("path/to/cropped_" + i, cropped_im)

In [ ]:
# Border images to make them square
os.chdir("path/to/dir")
for i in os.listdir():
    if re.search(r"whirls 14", i) != None:
        im = cv2.imread(i, -1)
        bordered_im = cv2.copyMakeBorder(im, 0, 0, 15, 15, cv2.BORDER_CONSTANT, value=0)
        cv2.imwrite("path/to/bordered_" + i, bordered_im)

In [ ]:
# Test UnetWeightMap
from unet_weight_map import UnetWeightMap

ma = cv2.imread("path/to/file.tif")
ma = cv2.cvtColor(ma, cv2.COLOR_BGR2GRAY)
ma = cv2.resize(ma, (608, 608), interpolation=cv2.INTER_NEAREST)
weight_map = UnetWeightMap(ma)

fig, ax = plt.subplots()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
im = ax.imshow(weight_map, cmap="hot")
fig.colorbar(im, cax=cax, orientation="vertical")
plt.show()

In [ ]:
# Conduct postprocessing to remove small segments
for i in os.listdir('path/to/dir'):
    pred = io.imread(i, plugin="tifffile")
    parasite_labels = measure.label(pred)
    for p in range(len(np.unique(parasite_labels))):
        if np.count_nonzero(parasite_labels == p) < 50:
            parasite_labels[parasite_labels == p] = 0
    parasite_labels[parasite_labels != 0] = 1
    parasite_labels = parasite_labels.astype("float32")
    io.imsave("path/to/file/" + i, parasite_labels, plugin="tifffile")

In [ ]:
# Threshold
pred_list = os.listdir('path/to/dir')
for i in pred_list:
    pred = io.imread(i, plugin="tifffile")
    pred[pred >= 0.5] = 1
    pred[pred < 0.5] = 0
    io.imsave("path/to/file/" + i, pred, plugin="tifffile")

In [ ]:
# Evaluation metrics
ma = io.imread("path/to/file.tif", plugin="tifffile")
ma = np.reshape(ma, (ma.shape[0], ma.shape[1])).astype(int)
pred = io.imread("path/to/file.tif", plugin="tifffile")
pred = np.reshape(pred, (pred.shape[0], pred.shape[1])).astype(int)

score = metrics.variation_of_information(ma, pred)

In [ ]:
# Evaluation metrics
groundtruth_list, prediction_list = [] # list images here for further evaluation
true_neg, false_pos, false_neg, true_pos = []
for i in range(len(groundtruth_list)):
    tn, fp, fn, tp = confusion_matrix(groundtruth_list[i].ravel(), prediction_list[i].ravel()).ravel()
    true_neg.append(tn)
    false_pos.append(fp)
    false_neg.append(fn)
    true_pos.append(tp)
true_neg = np.sum(true_neg)
false_pos = np.sum(false_pos)
false_neg = np.sum(false_neg)
true_pos = np.sum(true_pos)

# Accuracy
total = true_pos + true_neg + false_pos + false_neg
accuracy = (true_pos + true_neg) / total

# F1 score (Dice coefficient)
f1_score = (2 * true_pos) / ((2 * true_pos) + false_pos + false_neg)

precision = true_pos / (true_pos + false_pos)
recall = true_pos / (true_pos + false_neg)
f1_score2 = (2 * precision * recall) / (precision + recall)